# Performing the Calculation

Now that we have performed the transformation on the boavizta files, we can perform the calculation with an accurate database.<br><br>
The other data that goes into the calculation you will need to fill in here, such as:
- Number of devices refurbished by type
- Yearly carbon emissions for your facility
- Pickup location and sales location

In [ ]:
import pandas as pd

## 1 Carbon benefit per product

We'll import the boavizta CSV that we made:

In [ ]:
df_boavizta = pd.read_csv('CSVs/boavizta-product-averages.csv')

Now, assign below the number of each subcategory you wish to calculate for:

In [ ]:
All_in_One_24_greater = 0
All_in_One_24_less = 0
Average_All_in_One = 0
Average_Desktop = 0
Average_Laptop = 0
Average_Monitor = 0
Desktop_SFF = 0
Desktop_Tower = 0
Desktop_USDT = 0
Laptop_14_greater = 0
Laptop_14_lesser = 0
Monitor_33_greater = 0
Monitor_33_lesser = 0
Printer = 0
Smartphone = 0
Tablet = 0
Workstation = 0

Now, we need to assign all of the above in the dataframe:

In [ ]:
df_benefits = df_boavizta.copy()
df_benefits.set_index('subcategory',inplace=True)
df_benefits.loc['All-in-One 24+', 'counts']=All_in_One_24_greater
df_benefits.loc['All-in-One <24', 'counts']=All_in_One_24_less
df_benefits.loc['Average All-in-One', 'counts']=Average_All_in_One
df_benefits.loc['Average Desktop', 'counts']=Average_Desktop
df_benefits.loc['Average Laptop', 'counts']=Average_Laptop
df_benefits.loc['Average Monitor', 'counts']=Average_Monitor
df_benefits.loc['Desktop SFF','counts']=Desktop_SFF
df_benefits.loc['Desktop Tower','counts']=Desktop_Tower
df_benefits.loc['Desktop USDT','counts']=Desktop_USDT
df_benefits.loc['Laptop 14+','counts']=Laptop_14_greater
df_benefits.loc['Laptop <14','counts']=Laptop_14_lesser
df_benefits.loc['Monitor 33+','counts']=Monitor_33_greater
df_benefits.loc['Monitor <33','counts']=Monitor_33_lesser
df_benefits.loc['Printer','counts']=Printer
df_benefits.loc['Smartphone','counts']=Smartphone
df_benefits.loc['Tablet','counts']=Tablet
df_benefits.loc['Workstation','counts']=Workstation

Things that go in the calc:
- Transport emissions
- Refurbishing

## 2 Refurbishing Cost

Assign your organisation's scope 1 and 2 emissions in kg CO2e to the below variables:

In [ ]:
scope_1 = 0
scope_2 = 0
scope_1_and_2 = scope_1 + scope_2

Assign the total number of assets per year:

In [ ]:
total_assets = 0

Once we have this, we can use the following function which gets the total refurb cost:

In [ ]:
refurb_per_asset = scope_1_and_2/total_assets

Then, you can find the refurb cost by taking the count of the outputs in the calculation<br>
(Here I'm assuming that this is a calculation for a customer rather than yourself. Otherwise, you could use the total assets figure.)

In [ ]:
assets_processed = df_benefits['counts'].sum()
assets_processed

## 3 Transport Cost

We'll use the google maps api to calculate the distances.<br>You'll need to enter your Google Maps API key below.<br>You can make a key here: https://developers.google.com/maps and they are free up to $200 worth of calls. 

In [1]:
import googlemaps
#gmaps = googlemaps.Client(key='UNCOMMENT AND ENTER YOUR API KEY HERE')

You'll need to assign the following locations:

In [ ]:
pickup_location = 'Address that the asset came from to get to the facility'
facility_location = 'Address that the '
delivery_location = 